In [1]:
import yaml
from api.source.operators.controller import Controller

In [4]:
with open("api\configs\pipeline.yml", "r") as file:
    config = yaml.safe_load(file)

In [5]:
controller_instance = Controller(config)

In [6]:
controller_instance.process_video()

In [16]:
controller_instance.switch_model("faster_rcnn")

In [15]:
controller_instance.switch_camera("traffic_vn")